# FIRST TIMER AGENTIC AI

# LangSmith

for model, chain, agent maintaining and inspection

In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## Tavily

as a search engine tool

In [2]:
os.environ["TAVILY_API_KEY"] = getpass.getpass()

## Defining tools

We first need to create the tools we want to use. Our main tool of choice will be Tavily - a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as tool.



In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]



Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1739339778, 'localtime': '2025-02-11 21:56'}, 'current': {'last_updated_epoch': 1739339100, 'last_updated': '2025-02-11 21:45', 'temp_c': 8.9, 'temp_f': 48.0, 'is_day': 0, 'condition': {'text': 'Light rain', 'icon': '//cdn.weatherapi.com/weather/64x64/night/296.png', 'code': 1183}, 'wind_mph': 6.0, 'wind_kph': 9.7, 'wind_degree': 235, 'wind_dir': 'SW', 'pressure_mb': 1015.0, 'pressure_in': 29.98, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 73, 'cloud': 100, 'feelslike_c': 7.4, 'feelslike_f': 45.3, 'windchill_c': 6.9, 'windchill_f': 44.3, 'heatindex_c': 9.1, 'heatindex_f': 48.3, 'dewpoint_c': 6.2, 'dewpoint_f': 43.2, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 8.5, 'gust_kph': 13.6}}"}, {'url': 'https://world-weather.info/forecast/usa/s

In [4]:
## Using language models

import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("llama3-8b-8192", model_provider="groq")

you can call the language model by passing in a list of messages. By default, the response is a `content` string

In [5]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="10 times 10 minus 50")])
response.content

'To evaluate this expression, we need to follow the order of operations (PEMDAS):\n\n1. Multiply 10 and 10: 10 × 10 = 100\n2. Subtract 50 from the result: 100 - 50 = 50\n\nSo the final answer is 50.'

We can now see what it is like to enable this model to do tool calling. In order to enable that we use .bind_tools to give the language model knowledge of these tools

In [7]:
model_with_tools = model.bind_tools(tools)

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the content field as well as the tool_calls field.

In [13]:
response = model_with_tools.invoke([HumanMessage(content="give me a laptop recommendation")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: The Dell XPS 13 is a great option for a laptop. It's a compact and lightweight device that's perfect for everyday use. It has a beautiful 4K touchscreen display, up to 16 hours of battery life, and a range of processor and storage options to choose from.

The HP Envy x360 is another excellent choice. It's a convertible laptop that can be used in tablet mode, making it great for watching movies or browsing the web on the go. It has a 15.6-inch touchscreen display, up to 14 hours of battery life, and a range of processor and storage options.

The Lenovo Yoga C940 is a great option for those who want a convertible laptop. It has a 14-inch touchscreen display, up to 15 hours of battery life, and a range of processor and storage options. It's also very lightweight and has a sleek design.

These are just a few examples of the many great laptops available. When choosing a laptop, consider your budget, the type of tasks you'll be using it for, and the size and weight you're comf

In [11]:
response = model_with_tools.invoke([HumanMessage(content="who is thomas alfa edinson")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

# sampe otw create agent

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'Thomas Alfa Edison'}, 'id': 'call_qpxz', 'type': 'tool_call'}]


We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

## Creating Agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using LangGraph to construct the agent. Currently, we are using a high level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.

Now, we can initialize the agent with the LLM and the tools.

Note that we are passing in the model, not model_with_tools. That is because create_react_agent will call .bind_tools for us under the hood.

In [10]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent

We can now run the agent with a few queries! Note that for now, these are all stateless queries (it won't remember previous interactions). Note that the agent will return the final state at the end of the interaction (which includes any inputs, we will see later on how to get only the outputs).

First up, let's see how it responds when there's no need to call a tool:

In [12]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='0225b007-8e30-4465-b260-fa5a2fabe7be'),
 AIMessage(content='Hi!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 1902, 'total_tokens': 1905, 'completion_time': 0.0025, 'prompt_time': 0.249502936, 'queue_time': -0.296746792, 'total_time': 0.252002936}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-710c68f6-89cc-481f-a65c-e66e0bc7ee36-0', usage_metadata={'input_tokens': 1902, 'output_tokens': 3, 'total_tokens': 1905})]

In [ ]:
from langchain_openai import ChatOpenAI